In [1]:
from google.colab import drive
drive.mount('drive')
import os

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
!ls
print()
!ls drive
print()
!ls drive/My\ Drive/M2_IAAA/TAL

arbre.py	     embd.vec		     neural_network.py	sample_data
automate.py	     embedding.py	     Oracle.py		util.py
ConstructAllTree.py  f2_tbp.fm		     PP.py		WordBuffer.py
create_dicos.py      Features.py	     Projectivite.py	Word.py
Dicos.py	     fr_gsd-ud-train.conllu  __pycache__
drive		     Main.py		     read_conllu.py

'My Drive'

arbre.py	     Dicos.py		Oracle.py	 util.py
automate.py	     embedding.py	PP.py		 WordBuffer.py
ConstructAllTree.py  Features.py	Projectivite.py  Word.py
create_dicos.py      Main.py		read_conllu.py
Data		     neural_network.py	README.md


In [3]:
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
import numpy as np
from numpy import argmax
from ConstructAllTree import *
from Features import *
from Oracle import *
from neural_network import create_neural_network_model
import os
import numpy as np






Using TensorFlow backend.


In [0]:
def get_mcd():
    mcd = (
        ('INDEX', 'INT'), ('FORM', 'INT'), ('LEMMA',
                                            'INT'), ('POS', 'SYM'), ('X1', 'INT'), ('MORPHO', 'INT'),
        ('GOV', 'SYM'), ('LABEL', 'SYM'), ('X2', 'SYM'), ('X3', 'SYM'))

    return mcd


def set_mcd(mcd):
    mcd = mcd


def get_xy(file_conllu, file_features, file_embedding=None):
    mcd = get_mcd()

    print("Chargement des arbres")
    obj_generateAlltree = ConstructAllTree(file_conllu, mcd, True)

    all_tree = obj_generateAlltree.get_allTreeProjectiviser()
    # print(all_tree[0].print_tree())
    print("Arbres charger : ", len(all_tree))

    print("Création du dataset")
    features = Features(file_features)
    i = 0
    for tree in all_tree:
        i += 1
        if i % 1000 == 0:
            print(i)
        # tree.print_tree()
        # if i != 43 and i != 61:
        A = Oracle(tree, features)
        A.run()

    print("Convertion du dataset")
    print("file_embedding : ", file_embedding)
    X, Y = features.get_Data_Set(file_embedding)
    """X_onehot = features.convert_datas_to_one_hot()

	Y_onehot = features.convert_labels_to_one_hot()"""
    return X, Y


def get_data(file_features, file_train_conllu, file_embedding=None):

    x_train, y_train = get_xy(file_train_conllu, file_features, file_embedding)

    return x_train, y_train


In [0]:
features_file = "drive/My Drive/M2_IAAA/TAL/Data/f2_tbp.fm"
conllu_file = "drive/My Drive/M2_IAAA/TAL/Data/fr_gsd-ud-train.conllu"
weight_embedding_file = "drive/My Drive/M2_IAAA/TAL/Data/embd__fr_50.vec"

In [0]:
x_train, y_train = get_data(features_file, conllu_file, weight_embedding_file)
# x_train,x_test,y_train,y_test = get_data("Data/f1_tbp.fm","test.txt","test.txt")
print("x_train=", x_train.shape)
print("Y_train=", y_train.shape)
print("start_train")
input_dim = x_train.shape[1]
print("input_dim= ", input_dim)
nb_class = y_train.shape[1]
print("nb_class= ", nb_class)


Chargement des arbres
Arbres charger :  14554
Création du dataset
Features prise en compte :  [('Pile', 0, 'FORM'), ('Buffer', 0, 'FORM'), ('Pile', 0, 'POS'), ('Pile', 0, 'LEMMA'), ('Pile', 0, 'MORPHO'), ('Buffer', 0, 'POS'), ('Buffer', 0, 'LEMMA'), ('Buffer', 0, 'MORPHO'), ('Buffer', -1, 'POS'), ('Buffer', 1, 'POS'), 'DIST']
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [0]:
model1 = create_neural_network_model(nb_class, input_dim)
# Train the model, iterating on the data in batches of 32 samples
model.fit(x_train, y_train, epochs=1000)



In [0]:
model.save('model_f2.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model"